In [1]:
import heapq

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost from start to node
        self.h = 0  # Heuristic cost to goal
        self.f = 0  # Total cost

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    # Using Manhattan distance as heuristic
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid, start, goal):
    open_list = []
    closed_set = set()

    start_node = Node(start)
    goal_node = Node(goal)

    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.position == goal:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # Return reversed path

        closed_set.add(current_node.position)

        for new_position in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # Right, Down, Left, Up
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Check if the node is within the grid bounds
            if (0 <= node_position[0] < len(grid)) and (0 <= node_position[1] < len(grid[0])):
                # Check if it's a free space (1) and not in the closed set
                if grid[node_position[0]][node_position[1]] == 1 and node_position not in closed_set:
                    child_node = Node(node_position, current_node)
                    child_node.g = current_node.g + 1
                    child_node.h = heuristic(child_node.position, goal_node.position)
                    child_node.f = child_node.g + child_node.h

                    if add_to_open(open_list, child_node):
                        heapq.heappush(open_list, child_node)

    return None  # No path found

def add_to_open(open_list, child_node):
    for node in open_list:
        if child_node.position == node.position and child_node.g > node.g:
            return False
    return True

# Example grid (1 = free, 0 = obstacle)
grid = [
    [1, 1, 1, 1, 1],  # Row 0
    [0, 0, 1, 0, 0],  # Row 1
    [1, 1, 1, 1, 1],  # Row 2
    [0, 0, 0, 0, 0],  # Row 3
    [1, 1, 1, 1, 1],  # Row 4
]

# Define source and destination for each bot
bots = [
    {'start': (0, 0), 'goal': (2, 3)},  # Bot 0
    {'start': (0, 4), 'goal': (2, 1)},  # Bot 1
]

# Generate paths for each bot
paths = []
for bot in bots:
    path = a_star_search(grid, bot['start'], bot['goal'])
    paths.append(path)

# Print the paths for each bot
for index, path in enumerate(paths):
    print(f"Path for Bot {index}: {path}")

def get_direction(current, next):
    if next[0] == current[0] and next[1] > current[1]:
        return "right"
    elif next[0] == current[0] and next[1] < current[1]:
        return "left"
    elif next[1] == current[1] and next[0] > current[0]:
        return "forward"
    elif next[1] == current[1] and next[0] < current[0]:
        return "backward"
    else:
        return "staying in place"

# Initialize positions for each bot
positions = [path[0] for path in paths]  # Start at the first position of each bot
final_paths = [[], []]  # To track the final path of each bot

# Initialize indices for both bots
i = 0  # Index for bot 0
j = 0  # Index for bot 1

# Start the movement process until both bots reach their final destination
while i < len(paths[0]) or j < len(paths[1]):
    next_positions = {}  # To track the next positions of bots
    collisions = set()   # To track positions where potential collisions will happen

    # First pass: Check next positions for collisions
    for bot in [0, 1]:  # Use explicit bot indices
        if bot == 0 and i < len(paths[bot]):
            next_position = paths[bot][i]
        elif bot == 1 and j < len(paths[bot]):
            next_position = paths[bot][j]
        else:
            continue

        # Check if another bot is trying to move to the same position
        if next_position in next_positions:
            print(f"Potential collision detected at {next_position} between bot {bot} and bot {next_positions[next_position]}")
            collisions.add(next_position)  # Mark this position for a potential collision
        else:
            next_positions[next_position] = bot  # Record this bot's intended position

    # Second pass: Move bots, handle potential collisions
    for bot in [0, 1]:
        if bot == 0 and i < len(paths[bot]):
            current_position = paths[bot][i]
        elif bot == 1 and j < len(paths[bot]):
            current_position = paths[bot][j]
        else:
            continue

        # If there's a collision at the next position
        if current_position in collisions:
            other_bot = next_positions[current_position]

            # The bot with the lower index moves to the collision point while the other bot stays
            if bot == 0:
                print(f"Bot {bot} moves to {current_position} ({get_direction(positions[bot], current_position)})")
                positions[bot] = current_position  # Update position
                final_paths[bot].append(current_position)  # Track the position
                i += 1  # Move to next position for Bot 0
            else:
                # Bot 1 should stay in its previous position
                print(f"Bot {bot} stays at {positions[bot]}")
                final_paths[bot].append(positions[bot])  # Track the position
                continue  # Skip incrementing j for Bot 1
        else:
            # No collision, move the bot normally
            print(f"Bot {bot} moves to {current_position} ({get_direction(positions[bot], current_position)})")
            positions[bot] = current_position  # Update position
            final_paths[bot].append(current_position)  # Track the position
            if bot == 0:
                i += 1  # Move to next position for Bot 0
            else:
                j += 1  # Move to next position for Bot 1

# Display final paths after all movements
print("\nFinal paths:")
print(f"Bot 0 final path: {final_paths[0]}")
print(f"Bot 1 final path: {final_paths[1]}")


Path for Bot 0: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3)]
Path for Bot 1: [(0, 4), (0, 3), (0, 2), (1, 2), (2, 2), (2, 1)]
Bot 0 moves to (0, 0) (staying in place)
Bot 1 moves to (0, 4) (staying in place)
Bot 0 moves to (0, 1) (right)
Bot 1 moves to (0, 3) (left)
Potential collision detected at (0, 2) between bot 1 and bot 0
Bot 0 moves to (0, 2) (right)
Bot 1 stays at (0, 3)
Bot 0 moves to (1, 2) (forward)
Bot 1 moves to (0, 2) (left)
Bot 0 moves to (2, 2) (forward)
Bot 1 moves to (1, 2) (forward)
Bot 0 moves to (2, 3) (right)
Bot 1 moves to (2, 2) (forward)
Bot 1 moves to (2, 1) (left)

Final paths:
Bot 0 final path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3)]
Bot 1 final path: [(0, 4), (0, 3), (0, 3), (0, 2), (1, 2), (2, 2), (2, 1)]
